In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import logging
import sys
import pickle
from pathlib import Path

import torch
import wandb
import optuna
from optuna.trial import TrialState

from config import Environment, TrainConfig
from denoising.train import prepare_training
from denoising.utils import seed_everything
from denoising.models.utils import count_parameters

In [16]:
CWD = Path.cwd()
env = Environment(_env_file=CWD / '../env')
wandb.login(key=env.wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/d.nesterov/.netrc


True

In [37]:
def define_train_cfg(trial) -> TrainConfig:
    n_layers = trial.suggest_int('n_layers', 1, 20, log=True)
    hidden_channels = trial.suggest_categorical('hidden_channels', [4, 8, 16, 32])
    n_modes = trial.suggest_categorical('n_modes', [2, 4, 8, 16, 32])
    lifting_channel_ratio = trial.suggest_categorical('lifting_channel_ratio', [2, 4, 8, 16, 32])
    projection_channel_ratio = trial.suggest_categorical('projection_channel_ratio', [2, 4, 8, 16, 32])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    print(
        f'Running with {n_layers=}, {hidden_channels=}, {n_modes=}, {lifting_channel_ratio=}, {projection_channel_ratio=} {lr=}'
    )

    cfg = TrainConfig(
        # Datasets params
        train_dset='mri_pm_train',
        test_dset='mri_pm_test',
        train_batch_size=32,
        test_batch_size=32,
        # Model params
        name_model='mri-fno',
        cfg_fno={
            'n_modes': (n_modes, n_modes),
            'in_channels': 1,
            'hidden_channels': hidden_channels,
            'lifting_channel_ratio': lifting_channel_ratio,
            'projection_channel_ratio': projection_channel_ratio,
            'out_channels': 1,
            'factorization': 'dense',
            'n_layers': n_layers,
            'rank': 0.42,
            # 'spectral': 'hartley',
        },
        # Run params
        random_seed=42,
        device='cuda:2',
        run_name='Run optuna',
        # Train params
        n_epochs=3,
        lr=lr,
        verbose=True,
    )

    return cfg


def objective(trial):
    cfg = define_train_cfg(trial)
    trainer, train_kwargs, _ = prepare_training(env, cfg)

    if count_parameters(trainer.model) > 5_000_000:
        print('Pruned by model params')
        raise optuna.exceptions.TrialPruned

    seed_everything(cfg.random_seed)

    trial_obj = 'test_h1'
    metrics = trainer.train(trial=trial, trial_obj=trial_obj, **train_kwargs)

    return float(metrics[trial_obj])

In [38]:
run_name = 'fno'
study_name = f'{run_name}-optuna'
sampler_path = Path(f'./{run_name}-sampler')

In [41]:
# init trials storage and sampler pickle
optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
storage_name = 'sqlite:///{}.db'.format(study_name)
restored_sampler = None
if sampler_path.exists():
    print(f'Restore sampler from path: {sampler_path}')
    restored_sampler = pickle.load(Path.open(sampler_path, 'rb'))

# create new study or restore
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    direction='minimize',
    sampler=restored_sampler,
    load_if_exists=True,
)

if not sampler_path.exists():
    print(f'Caching sampler in: {sampler_path}')
    pickle.dump(study.sampler, Path.open(sampler_path, 'wb'))

# run optimization
study.optimize(objective, n_trials=100, timeout=1200)

[I 2025-04-19 18:06:42,878] Using an existing study with name 'fno-optuna' instead of creating a new one.


Restore sampler from path: fno-sampler
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study with name 'fno-optuna' instead of creating a new one.
Using an existing study w

[I 2025-04-19 18:07:18,678] Trial 15 pruned. 


Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Trial 15 pruned. 
Running with n_layers=11, hidden_channels=4, n_modes=16, lifting_channel_ratio=32, projection_channel_ratio=4 lr=0.030553728606549646
Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test 

[I 2025-04-19 18:08:25,412] Trial 16 finished with value: 0.10326272994279861 and parameters: {'n_layers': 11, 'hidden_channels': 4, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 4, 'lr': 0.030553728606549646}. Best is trial 4 with value: 0.0870702937245369.


...

Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Trial 26 pruned. 
Running with n_layers=20, hidden_channels=16, n_modes=16, lifting_channel_ratio=32, projection_channel_ratio=2 lr=0.0026694812240906154
Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_tes

[I 2025-04-19 18:25:54,723] Trial 27 finished with value: 0.09219709783792496 and parameters: {'n_layers': 20, 'hidden_channels': 16, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.0026694812240906154}. Best is trial 4 with value: 0.0870702937245369.


Trial 27 finished with value: 0.09219709783792496 and parameters: {'n_layers': 20, 'hidden_channels': 16, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.0026694812240906154}. Best is trial 4 with value: 0.0870702937245369.
Trial 27 finished with value: 0.09219709783792496 and parameters: {'n_layers': 20, 'hidden_channels': 16, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.0026694812240906154}. Best is trial 4 with value: 0.0870702937245369.
Trial 27 finished with value: 0.09219709783792496 and parameters: {'n_layers': 20, 'hidden_channels': 16, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.0026694812240906154}. Best is trial 4 with value: 0.0870702937245369.
Trial 27 finished with value: 0.09219709783792496 and parameters: {'n_layers': 20, 'hidden_channels': 16, 'n_modes': 16, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.0026694812240906154}. Best is tria

[I 2025-04-19 18:28:42,781] Trial 28 finished with value: 0.08233946561813354 and parameters: {'n_layers': 15, 'hidden_channels': 16, 'n_modes': 32, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.006055187761870968}. Best is trial 28 with value: 0.08233946561813354.


Trial 28 finished with value: 0.08233946561813354 and parameters: {'n_layers': 15, 'hidden_channels': 16, 'n_modes': 32, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.006055187761870968}. Best is trial 28 with value: 0.08233946561813354.
Trial 28 finished with value: 0.08233946561813354 and parameters: {'n_layers': 15, 'hidden_channels': 16, 'n_modes': 32, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.006055187761870968}. Best is trial 28 with value: 0.08233946561813354.
Trial 28 finished with value: 0.08233946561813354 and parameters: {'n_layers': 15, 'hidden_channels': 16, 'n_modes': 32, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.006055187761870968}. Best is trial 28 with value: 0.08233946561813354.
Trial 28 finished with value: 0.08233946561813354 and parameters: {'n_layers': 15, 'hidden_channels': 16, 'n_modes': 32, 'lifting_channel_ratio': 32, 'projection_channel_ratio': 2, 'lr': 0.006055187761870968}. Best is tr

In [42]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print('Study statistics: ')
print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of complete trials: ', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Study statistics: 
  Number of finished trials:  29
  Number of pruned trials:  9
  Number of complete trials:  20
Best trial:
  Value:  0.08233946561813354
  Params: 
    n_layers: 15
    hidden_channels: 16
    n_modes: 32
    lifting_channel_ratio: 32
    projection_channel_ratio: 2
    lr: 0.006055187761870968


In [31]:
# FNO
# Study statistics: 
#   Number of finished trials:  29
#   Number of pruned trials:  9
#   Number of complete trials:  20
# Best trial:
#   Value:  0.08233946561813354
#   Params: 
#     n_layers: 15
#     hidden_channels: 16
#     n_modes: 32
#     lifting_channel_ratio: 32
#     projection_channel_ratio: 2
#     lr: 0.006055187761870968

# HNO
# Study statistics: 
#   Number of finished trials:  55
#   Number of pruned trials:  31
#   Number of complete trials:  24
# Best trial:
#   Value:  0.08180370926856995
#   Params: 
#     n_layers: 10
#     hidden_channels: 16
#     n_modes: 16
#     lifting_channel_ratio: 32
#     projection_channel_ratio: 8
#     lr: 0.00433647012426727

In [16]:
# HNO-v2, test_h1
# Study statistics:
#   Number of finished trials:  25
#   Number of pruned trials:  16
#   Number of complete trials:  9
# Best trial:
#   Value:  0.10859087109565735
#   Params:
#     n_layers: 2
#     hidden_channels: 41
#     n_modes: 8
#     lifting_channel_ratio: 6
#     projection_channel_ratio: 32
#     lr: 0.0026019787737744096


# HNO-v2, test_l2

# Study statistics:
#   Number of finished trials:  31
#   Number of pruned trials:  13
#   Number of complete trials:  17
# Best trial:
#   Value:  0.043848007917404175
#   Params:
#     n_layers: 3
#     hidden_channels: 49
#     n_modes: 16
#     lifting_channel_ratio: 6
#     projection_channel_ratio: 12
#     lr: 0.0074820412780186325